In [ ]:
using MIPVerify
using Gurobi
using JuMP
using Images
using Printf
function print_summary(d::Dict)
    # Helper function to print out output
    obj_val = JuMP.objective_value(d[:Model])
    solve_time = JuMP.solve_time(d[:Model])
    println("Objective Value: $(obj_val), Solve Time: $(@sprintf("%.2f", solve_time))")
end

function view_diff(diff::Array{<:Real, 2})
    n = 1001
    colormap("RdBu", n)[ceil.(Int, (diff .+ 1) ./ 2 .* n)]
end

In [ ]:
struct VerifyInfo
    index::Int
    time::Float64
    adv_found::Bool
    adv_not_found::Bool
    verified::Bool
    verified_res::Int
end

In [ ]:
##验证784输入维度网络的鲁棒性

In [ ]:
function robust_verify_mnist784()
    misclassified = 0
    result = VerifyInfo[]
    round_error_class=Int[]
    mnist = MIPVerify.read_datasets("MNIST")
    nn784 = MIPVerify.get_example_network_params("F16MNIST24")
    for index in 1:5513
        total_time = 0 
        try
            sample_image = Float16.(MIPVerify.get_image(mnist.test.images, index))
            sample_label = MIPVerify.get_label(mnist.test.labels, index)
            predicted_output = sample_image |> nn784
            predicted_output_index = argmax(predicted_output)
            if predicted_output_index != sample_label + 1
                #println("Sample $index misclassified.")
                misclassified = misclassified+1
                continue
            end
            label_index = [1,2,3,4,5,6,7,8,9,10]    
            target_index = [i for i in label_index if i!=predicted_output_index]
            @assert (predicted_output_index - 1) == sample_label
            total_time = @elapsed begin
                d = MIPVerify.robustness_checking_verification(
                    nn784, 
                    sample_image, 
                    target_index, 
                    Gurobi.Optimizer,
                    #OutputFlag=0, #prevents any output from being printed out
                    Dict("OutputFlag" => 0),
                    pp = MIPVerify.LInfNormBoundedPerturbationFamily(0.15),
                    norm_order = Inf,
                    tightening_algorithm = lp,
                    margin=floatmin(Float16)
                )
            end
                perturbed_sample_image = Float16.(value.(d[:PerturbedInput]))
                perturbation_predicted_output = perturbed_sample_image |> nn784
                perturbation_predicted_index = argmax(perturbation_predicted_output)

            if(perturbation_predicted_index==predicted_output_index)
                #发现虚假反例
                false_adv_res=VerifyInfo(index,total_time,false,false,false,0)
                push!(result,false_adv_res)
            else
                #发现真实反例
                true_adv_res=VerifyInfo(index,total_time,true,false,true,-1)
                push!(result,true_adv_res)
            end

        catch e
            #println("处理样本 $index 时出错: $e")
            verified_robust = VerifyInfo(index,total_time,false,true,true,1)
            push!(result,verified_robust)
        end
    end
    run_statistics = Dict(
        :result => result,
        :misclassified => misclassified,
        :round_error_class => round_error_class
    )
    return run_statistics
end
 

In [ ]:
return784=robust_verify_mnist784()

In [ ]:
res784=return784[:result]
# 创建一个空字典
statistics_dict = Dict{String, Int}()

# 统计 tp_adv
tp_adv = count(x -> x.adv_found == true, res784)
statistics_dict["tp_adv"] = tp_adv

# 统计 tf_adv
tf_adv = count(x -> x.adv_found == false, res784)
statistics_dict["tf_adv"] = tf_adv

# 统计 verified_t
verified_t = count(x -> x.verified == true && x.verified_res == 1, res784)
statistics_dict["verified_t"] = verified_t

# 统计 verified_f
verified_f = count(x -> x.verified == true && x.verified_res == -1, res784)
statistics_dict["verified_f"] = verified_f

verified_unknow = count(x -> x.verified == false && x.verified_res == 0, res784)
statistics_dict["verified_unknow"] = verified_unknow 

# 统计样本总数total
statistics_dict["total_sample"] = 5513-return784[:misclassified]

# 打印字典
println(statistics_dict)
